In [2]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.script_run_config import ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies

In [3]:
ws = Workspace.from_config()
experiment_name = '<YOUR EXPERIMENT NAME>'
experiment = Experiment(ws, experiment_name)

amlcompute_cluster_name = '<YOUR_AML_CLUSTER_NAME>'
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    compute_target = cts[amlcompute_cluster_name]

print('Checking cluster status...')
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

Checking cluster status...
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [14]:
cd = CondaDependencies.create(conda_packages=["pyodbc"])

conda_run_config = RunConfiguration(framework="python", conda_dependencies=cd)
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = 'azuremlpyimg'
conda_run_config.environment.docker.base_image_registry.address = 'sercohs07c67206.azurecr.io'
conda_run_config.environment.python.conda_dependencies = cd

src = ScriptRunConfig(
    source_directory='./', 
    script='connect_to_sqldw.py', 
    run_config=conda_run_config,
) 
run = experiment.submit(config=src)
